# Extremal linkage networks

This notebook contains code accompanying the paper [extremal linkage networks](https://arxiv.org/abs/1904.01817). 

We first implement the network dynamics and then rely on [TikZ](https://github.com/pgf-tikz/pgf) for visualization.

## The Model

We define a random network on an infinite set of layers, each consisting of $N \ge 1$ nodes. The node $i \in  \{0, \dots, N - 1\}$ in layer $h \in \mathbb Z$ has a fitness $F_{i, h}$, where we assume the family $\{F_{i, h}\}_{i \in \{0, \dots, N - 1\}, h \in \mathbb Z}$ to be independent and identically distributed (i.i.d.).


Then, the number of nodes on layer $h+1$ that are visible for the $i$th node in layer $h$, which we call the *scope* of $(i,h)$, is given by $\varphi(F_{i, h}) \wedge N$, where
\begin{equation}\label{eqPhiDef}
        \varphi(f) = 1 + 2 \lceil f \rceil.
\end{equation}


Now, $(i, h)$ connects to precisely one visible node $(j, h+1)$ in layer $h+1$, namely the one of maximum fitness.  In other words,
$$F_{j, h+1} = \max_{j':\, d_N(i, j') \le \lceil F_{i, h}\rceil}F_{j', h+1}.$$


Henceforth, we assume the fitnesses to follow a Fréchet distribution with tail index 1. That is,
$$\mathbb P(F \le s) = \exp(-s^{-1}).$$


## Simulation of Network Dynamics

In [ ]:
def simulate_network(hrange = 250,
                     layers = 6):
    """Simulation of the network model
    # Arguments
        hrange: horizontal range of the network
        layers: number of layers
       
    # Result
        fitnesses and selected edge
    """
    #generate fréchet distribution
    fits =  np.array([1/np.log(1/np.random.rand(hrange)) for _ in range(layers)]) 
    fits_int = 1+ np.array(fits, dtype = np.int)
    
    #determine possible neighbors
    neighbs = [[(idx + np.arange(-fit, fit + 1)) % hrange 
      for  idx, fit in enumerate(layer) ] 
     for layer in fits_int]

    #determine selected neighbor
    sel_edge = [[neighb[np.argmax(fit[neighb])] 
      for neighb in nb] 
     for (fit, nb) in zip(np.roll(fits, -1, 0), neighbs)]
    
    return fits, sel_edge

Now, we simulate the random network model as described above.

In [ ]:
import numpy as np

#seed
seed = 56
np.random.seed(seed)

fits, edges = simulate_network()

## Visualization

Now, plot the network in tikz.

In [ ]:
def plot_synapses(edges,
                  idxs = np.arange(102, 131),
                  layers = 4,
                  x_scale = .15,
                  node_scale = .5):
    """Plot relevant synapses
    # Arguments
        idxs: indexes of layer-0 node
        edges: edges in the linkage graph
        layers: number of layers to be plotted
        x_scale: scaling in x-direction
        node_scale: scaling of nodes
        
    # Result
        tikz representation of graph
    """
    result = []

    #horizontal range
    hrange = len(edges[0])

    #plot layer by layer
    for layer in range(layers):
        #plot points 
        result +=["\\fill ({0:1.2f}, {1:1.2f}) circle ({2:1.1f}pt);\n".format((idx % hrange) * x_scale, 
                                                                              layer,
                                                                              node_scale * np.log(fits)[layer, idx]) 
              for idx in idxs] 
        
        #plot edges
        string = "\\draw[line width = .5pt]"
        string += " ({0:1.2f}, {1:1.2f})--({2:1.2f}, {3:1.2f});\n"
        path_unordered = [string.format(idx  * x_scale,
                                        layer,
                                        edges[layer][idx] * x_scale,
                                        layer + 1) for idx in idxs]
        result += path_unordered

        #update indexes
        idxs = np.unique([edges[layer][idx] for idx in idxs])


    #plot points 
    result +=["\\fill ({0:1.2f}, {1:1.2f}) circle ({2:1.1f}pt);\n".format((idx % hrange) * x_scale, 
                                                                          layers,
                                                                          node_scale * np.log(fits)[layer + 1, idx]) 
              for idx in idxs] 

    

    tikz = ''.join(result)

    return '\\begin{tikzpicture}\n' + tikz + '\\end{tikzpicture}\n'

Finally, we write to a file.

In [ ]:
fname = 'coalesc.tex'

f = open(fname, "w")
f.write(plot_synapses(idxs,
                      edges))
f.close()

In [ ]:
!pdflatex evolFig.tex
